<a href="https://colab.research.google.com/github/Gcoles22/GCs-Room-Proposer/blob/main/Pace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- PACE MODULE 1 (V13): MULTI-FILE INGESTION, CLEANING, AND ANALYSIS ---

import pandas as pd
import numpy as np # Import numpy for NaN handling

# --- COMMON FUNCTIONS (Defined once) ---

# 1. Function to clean money values (removes '$', ',', converts to float)
def clean_money(value):
    if pd.isna(value):
        return 0.0
    clean_val = str(value).replace('$', '').replace(',', '').strip().replace('(', '-').replace(')', '')
    try:
        return float(clean_val)
    except ValueError:
        return 0.0

# 2. Function to standardize supplier names (specifically for Big 4)
def clean_supplier_name(name):
    if pd.isna(name):
        return "UNKNOWN" # Placeholder for missing names

    name_str = str(name).upper()

    if 'PWC' in name_str or 'PRICEWATERHOUSE' in name_str:
        return 'PwC (Consolidated)'
    elif 'DELOITTE' in name_str:
        return 'Deloitte (Consolidated)'
    elif 'ERNST & YOUNG' in name_str or 'EY' in name_str:
        return 'EY (Consolidated)'
    elif 'KPMG' in name_str:
        return 'KPMG (Consolidated)'
    else:
        return name

# --- MULTI-FILE INGESTION PROCESS ---

# Define the list of files you want to process
file_names = [
    'pwc_pace_data.csv',
    'deloitte_pace_data.csv',
    'kpmg_pace_data.csv',
    'ey_pace_data.csv'
]

all_processed_dfs = [] # This list will hold each cleaned DataFrame

print("--- Starting Multi-File Ingestion ---")

for file_name in file_names:
    print(f"\nProcessing file: {file_name}...")
    temp_df = None # Initialize temp_df
    try:
        # Try UTF-8 first
        temp_df = pd.read_csv(file_name, encoding='utf-8', skiprows=16, header=None)
        print("  Attempted UTF-8 encoding.")
    except UnicodeDecodeError:
        try:
            # If UTF-8 fails, try Latin-1
            temp_df = pd.read_csv(file_name, encoding='latin-1', skiprows=16, header=None)
            print("  UTF-8 failed, successfully tried Latin-1 encoding.")
        except Exception as e:
            print(f"❌ Error: Could not read '{file_name}' with UTF-8 or Latin-1: {e}")
            continue # Skip to next file if both fail
    except FileNotFoundError:
        print(f"❌ Error: File '{file_name}' not found. Please upload it to Colab.")
        continue # Skip to next file

    if temp_df is not None:
        try:
            # Manually set headers from the second row (index 1) of the raw loaded data
            temp_df.columns = temp_df.iloc[1]

            # Drop the first two rows (the NaN row and the header row)
            temp_df = temp_df[2:].reset_index(drop=True)

            print(f"✅ Success! Data loaded and headers assigned from {file_name}.")

            # Apply cleaning functions
            temp_df['Clean_Supplier'] = temp_df['Supplier Name'].apply(clean_supplier_name)
            if 'Value (AUD)' in temp_df.columns:
                temp_df['Clean_Value'] = temp_df['Value (AUD)'].apply(clean_money)
            else:
                print(f"❌ 'Value (AUD)' column not found in {file_name}. Assigning 0.0.")
                temp_df['Clean_Value'] = 0.0 # Assign a default to avoid further errors

            all_processed_dfs.append(temp_df)
            print(f"  Rows loaded and cleaned: {len(temp_df)}")

        except Exception as e:
            print(f"❌ Error processing data structure in '{file_name}': {e}")


# --- CONSOLIDATION AND DEDUPLICATION ---

if all_processed_dfs:
    df = pd.concat(all_processed_dfs, ignore_index=True)
    print(f"\n--- All files concatenated. Initial total rows: {len(df)} ---")

    # Remove duplicate contracts based on 'CN ID' (Contract Notice ID)
    original_rows = len(df)
    if 'CN ID' in df.columns:
        df.drop_duplicates(subset=['CN ID'], keep='first', inplace=True)
        deduplicated_rows = len(df)
        print(f"--- Deduplicated {original_rows - deduplicated_rows} rows. Remaining unique contracts: {deduplicated_rows} ---")
    else:
        print("⚠️ 'CN ID' column not found for deduplication. Skipping deduplication.")
        deduplicated_rows = original_rows

    print("\n✅ Final Consolidated Data loaded for analysis.")
    print(f"Total unique contracts: {deduplicated_rows}")
    print("Columns in final DataFrame:", list(df.columns))
    print("First 5 rows of consolidated data:")
    print(df.head())

else:
    print("\n❌ No data frames were successfully processed. PACE engine cannot proceed.")
    raise SystemExit("No data loaded to process.")

# --- PACE MODULE 2: CORE ANALYSIS ---

# Ensure Clean_Value is numeric (important for sum operations later)
df['Clean_Value'] = pd.to_numeric(df['Clean_Value'], errors='coerce').fillna(0)

# Analysis A: Total Spend by the Consolidated Big 4 Entities
# Define the Big 4 firms for consolidated reporting
big_4_firms = [
    'PwC (Consolidated)',
    'Deloitte (Consolidated)',
    'EY (Consolidated)',
    'KPMG (Consolidated)'
]

# Analysis B: The "Vague Description" Hunter (now smarter!)
vague_keywords = [
    'management advisory services', 'consulting service', 'strategic advice',
    'professional services', 'advisory services', 'business advisory services',
    'support services', 'review services', 'quality assurance',
    'advisory support', 'program management', 'capacity building',
    'evaluation services', 'transformation services', 'change management'
]

vague_pattern = '|'.join(vague_keywords)
keyword_vague_data = df[
    df['Title'].fillna('').str.lower().str.contains(vague_pattern, na=False)
]

short_titles_data = df[
    df['Title'].fillna('').apply(lambda x: len(str(x).split())) < 4
]

vague_data = pd.concat([keyword_vague_data, short_titles_data]).drop_duplicates(subset=['CN ID'])

total_vague = vague_data['Clean_Value'].sum()
vague_count = len(vague_data)


# Analysis C: The "Just Under Threshold" Hunter
threshold_min = 70000.00
threshold_max = 79999.99

just_under_threshold_data = df[
    (df['Clean_Value'] >= threshold_min) &
    (df['Clean_Value'] <= threshold_max)
]
total_just_under_threshold = just_under_threshold_data['Clean_Value'].sum()
count_just_under_threshold = len(just_under_threshold_data)

# Analysis D: The "Super Red Flag" Hunter (Vague AND Just Under Threshold)
super_red_flag_data = pd.merge(
    vague_data[['CN ID', 'Clean_Value']],
    just_under_threshold_data[['CN ID', 'Clean_Value']],
    on='CN ID',
    how='inner'
)
super_red_flag_count = len(super_red_flag_data)
super_red_flag_total = super_red_flag_data['Clean_Value_x'].sum()


# --- PACE MODULE 3: REPORTING ---
print("\n" + "="*40)
print("      PACE ENGINE - CONSOLIDATED REPORT      ")
print("="*40)

# Report for each of the Big 4 firms
total_big_4_spend = 0
for firm in big_4_firms:
    firm_data = df[df['Clean_Supplier'] == firm]
    firm_spend = firm_data['Clean_Value'].sum()
    total_big_4_spend += firm_spend
    if firm_spend > 0: # Only print if there was actual spend found for the firm
        print(f"💰 Total Spend with {firm:<22}: ${firm_spend:,.2f}")

print(f"\n📈 Grand Total Big 4 Spend (in this data): ${total_big_4_spend:,.2f}")
print("="*40)

print(f"🌫️  Contracts with Vague Descriptions:   {vague_count}")
print(f"💸 Total Value of Vague Contracts:      ${total_vague:,.2f}")
print(f"📉 Contracts Just Under $80k Threshold: {count_just_under_threshold}")
print(f"💵 Value of Just Under Threshold Contracts: ${total_just_under_threshold:,.2f}")
print(f"🚨 Super Red Flag (Vague & Under Threshold): {super_red_flag_count}")
print(f"💲 Value of Super Red Flag Contracts: ${super_red_flag_total:,.2f}")
print("="*40)

# --- Sample Outputs for each category ---

print("\n--- Sample of Vague Contracts Found: ---")
if not vague_data.empty:
    original_vague_contracts = df[df['CN ID'].isin(vague_data['CN ID'])]
    display_cols = ['Agency', 'Title', 'Value (AUD)', 'Supplier Name']
    actual_display_cols = [col for col in display_cols if col in original_vague_contracts.columns]
    print(original_vague_contracts[actual_display_cols].head(5))
else:
    print("No vague contracts found in this sample.")

print("\n--- Sample of 'Just Under Threshold' Contracts Found: ---")
if not just_under_threshold_data.empty:
    original_threshold_contracts = df[df['CN ID'].isin(just_under_threshold_data['CN ID'])]
    display_cols = ['Agency', 'Title', 'Value (AUD)', 'Supplier Name']
    actual_display_cols = [col for col in display_cols if col in original_threshold_contracts.columns]
    print(original_threshold_contracts[actual_display_cols].head(5))
else:
    print("No 'Just Under Threshold' contracts found in this sample.")

print("\n--- Sample of 'Super Red Flag' Contracts Found: ---")
if not super_red_flag_data.empty:
    original_super_red_flags = df[df['CN ID'].isin(super_red_flag_data['CN ID'])]
    display_cols = ['Agency', 'Title', 'Value (AUD)', 'Supplier Name']
    actual_display_cols = [col for col in display_cols if col in original_super_red_flags.columns]
    print(original_super_red_flags[actual_display_cols].head())
else:
    print("No 'Super Red Flag' contracts found in this sample.")


--- Starting Multi-File Ingestion ---

Processing file: pwc_pace_data.csv...
  Attempted UTF-8 encoding.
✅ Success! Data loaded and headers assigned from pwc_pace_data.csv.
  Rows loaded and cleaned: 155

Processing file: deloitte_pace_data.csv...
  UTF-8 failed, successfully tried Latin-1 encoding.
✅ Success! Data loaded and headers assigned from deloitte_pace_data.csv.
  Rows loaded and cleaned: 3298

Processing file: kpmg_pace_data.csv...
  UTF-8 failed, successfully tried Latin-1 encoding.
✅ Success! Data loaded and headers assigned from kpmg_pace_data.csv.
  Rows loaded and cleaned: 4977

Processing file: ey_pace_data.csv...
  Attempted UTF-8 encoding.
✅ Success! Data loaded and headers assigned from ey_pace_data.csv.
  Rows loaded and cleaned: 72

--- All files concatenated. Initial total rows: 8502 ---
--- Deduplicated 0 rows. Remaining unique contracts: 8502 ---

✅ Final Consolidated Data loaded for analysis.
Total unique contracts: 8502
Columns in final DataFrame: ['CN ID', 'T

In [ ]:
# --- PACE MODULE 1 (V13): MULTI-FILE INGESTION, CLEANING, AND ANALYSIS ---

import pandas as pd
import numpy as np # Import numpy for NaN handling

# --- COMMON FUNCTIONS (Defined once) ---

# 1. Function to clean money values (removes '$', ',', converts to float)
def clean_money(value):
    if pd.isna(value):
        return 0.0
    clean_val = str(value).replace('$', '').replace(',', '').strip().replace('(', '-').replace(')', '')
    try:
        return float(clean_val)
    except ValueError:
        return 0.0

# 2. Function to standardize supplier names (specifically for Big 4)
def clean_supplier_name(name):
    if pd.isna(name):
        return "UNKNOWN" # Placeholder for missing names

    name_str = str(name).upper()

    if 'PWC' in name_str or 'PRICEWATERHOUSE' in name_str:
        return 'PwC (Consolidated)'
    elif 'DELOITTE' in name_str:
        return 'Deloitte (Consolidated)'
    elif 'ERNST & YOUNG' in name_str or 'EY' in name_str:
        return 'EY (Consolidated)'
    elif 'KPMG' in name_str:
        return 'KPMG (Consolidated)'
    else:
        return name

# --- MULTI-FILE INGESTION PROCESS ---

# Define the list of files you want to process
file_names = [
    'pwc_pace_data.csv',
    'deloitte_pace_data.csv',
    'kmpg_pace_data.csv', # Corrected typo kmpg -> kpmg if it was there in file_names list
    'ey_pace_data.csv'
]

all_processed_dfs = [] # This list will hold each cleaned DataFrame

print("--- Starting Multi-File Ingestion ---")

for file_name in file_names:
    print(f"\nProcessing file: {file_name}...")
    temp_df = None # Initialize temp_df
    try:
        # Try UTF-8 first
        temp_df = pd.read_csv(file_name, encoding='utf-8', skiprows=16, header=None)
        print("  Attempted UTF-8 encoding.")
    except UnicodeDecodeError:
        try:
            # If UTF-8 fails, try Latin-1
            temp_df = pd.read_csv(file_name, encoding='latin-1', skiprows=16, header=None)
            print("  UTF-8 failed, successfully tried Latin-1 encoding.")
        except Exception as e:
            print(f"❌ Error: Could not read '{file_name}' with UTF-8 or Latin-1: {e}")
            continue # Skip to next file if both fail
    except FileNotFoundError:
        print(f"❌ Error: File '{file_name}' not found. Please upload it to Colab.")
        continue # Skip to next file

    if temp_df is not None:
        try:
            # Manually set headers from the second row (index 1) of the raw loaded data
            temp_df.columns = temp_df.iloc[1]

            # Drop the first two rows (the NaN row and the header row)
            temp_df = temp_df[2:].reset_index(drop=True)

            print(f"✅ Success! Data loaded and headers assigned from {file_name}.")

            # Apply cleaning functions
            temp_df['Clean_Supplier'] = temp_df['Supplier Name'].apply(clean_supplier_name)
            if 'Value (AUD)' in temp_df.columns:
                temp_df['Clean_Value'] = temp_df['Value (AUD)'].apply(clean_money)
            else:
                print(f"❌ 'Value (AUD)' column not found in {file_name}. Assigning 0.0.")
                temp_df['Clean_Value'] = 0.0 # Assign a default to avoid further errors

            all_processed_dfs.append(temp_df)
            print(f"  Rows loaded and cleaned: {len(temp_df)}")

        except Exception as e:
            print(f"❌ Error processing data structure in '{file_name}': {e}")


# --- CONSOLIDATION AND DEDUPLICATION ---

if all_processed_dfs:
    df = pd.concat(all_processed_dfs, ignore_index=True)
    print(f"\n--- All files concatenated. Initial total rows: {len(df)} ---")

    # Remove duplicate contracts based on 'CN ID' (Contract Notice ID)
    original_rows = len(df)
    if 'CN ID' in df.columns:
        df.drop_duplicates(subset=['CN ID'], keep='first', inplace=True)
        deduplicated_rows = len(df)
        print(f"--- Deduplicated {original_rows - deduplicated_rows} rows. Remaining unique contracts: {deduplicated_rows} ---")
    else:
        print("⚠️ 'CN ID' column not found for deduplication. Skipping deduplication.")
        deduplicated_rows = original_rows

    print("\n✅ Final Consolidated Data loaded for analysis.")
    print(f"Total unique contracts: {deduplicated_rows}")
    print("Columns in final DataFrame:", list(df.columns))
    print("First 5 rows of consolidated data:")
    print(df.head())

else:
    print("\n❌ No data frames were successfully processed. PACE engine cannot proceed.")
    raise SystemExit("No data loaded to process.")

# --- PACE MODULE 2: CORE ANALYSIS ---

# Ensure Clean_Value is numeric (important for sum operations later)
df['Clean_Value'] = pd.to_numeric(df['Clean_Value'], errors='coerce').fillna(0)

# Analysis A: Total Spend by the Consolidated Big 4 Entities
# Define the Big 4 firms for consolidated reporting
big_4_firms = [
    'PwC (Consolidated)',
    'Deloitte (Consolidated)',
    'EY (Consolidated)',
    'KPMG (Consolidated)'
]

# Analysis B: The "Vague Description" Hunter (now smarter!)
vague_keywords = [
    'management advisory services', 'consulting service', 'strategic advice',
    'professional services', 'advisory services', 'business advisory services',
    'support services', 'review services', 'quality assurance',
    'advisory support', 'program management', 'capacity building',
    'evaluation services', 'transformation services', 'change management'
]

vague_pattern = '|'.join(vague_keywords)
keyword_vague_data = df[
    df['Title'].fillna('').str.lower().str.contains(vague_pattern, na=False)
]

short_titles_data = df[
    df['Title'].fillna('').apply(lambda x: len(str(x).split())) < 4
]

vague_data = pd.concat([keyword_vague_data, short_titles_data]).drop_duplicates(subset=['CN ID'])

total_vague = vague_data['Clean_Value'].sum()
vague_count = len(vague_data)


# Analysis C: The "Just Under Threshold" Hunter
threshold_min = 70000.00
threshold_max = 79999.99

just_under_threshold_data = df[
    (df['Clean_Value'] >= threshold_min) &
    (df['Clean_Value'] <= threshold_max)
]
total_just_under_threshold = just_under_threshold_data['Clean_Value'].sum()
count_just_under_threshold = len(just_under_threshold_data)

# Analysis D: The "Super Red Flag" Hunter (Vague AND Just Under Threshold)
super_red_flag_data = pd.merge(
    vague_data[['CN ID', 'Clean_Value']],
    just_under_threshold_data[['CN ID', 'Clean_Value']],
    on='CN ID',
    how='inner'
)
super_red_flag_count = len(super_red_flag_data)
super_red_flag_total = super_red_flag_data['Clean_Value_x'].sum()


# --- PACE MODULE 3: REPORTING ---
print("\n" + "="*40)
print("      PACE ENGINE - CONSOLIDATED REPORT      ")
print("="*40)

# Report for each of the Big 4 firms
total_big_4_spend = 0
for firm in big_4_firms:
    firm_data = df[df['Clean_Supplier'] == firm]
    firm_spend = firm_data['Clean_Value'].sum()
    total_big_4_spend += firm_spend
    if firm_spend > 0: # Only print if there was actual spend found for the firm
        print(f"💰 Total Spend with {firm:<22}: ${firm_spend:,.2f}")

print(f"\n📈 Grand Total Big 4 Spend (in this data): ${total_big_4_spend:,.2f}")
print("="*40)

print(f"🌫️  Contracts with Vague Descriptions:   {vague_count}")
print(f"💸 Total Value of Vague Contracts:      ${total_vague:,.2f}")
print(f"📉 Contracts Just Under $80k Threshold: {count_just_under_threshold}")
print(f"💵 Value of Just Under Threshold Contracts: ${total_just_under_threshold:,.2f}")
print(f"🚨 Super Red Flag (Vague & Under Threshold): {super_red_flag_count}")
print(f"💲 Value of Super Red Flag Contracts: ${super_red_flag_total:,.2f}")
print("="*40)

# --- Sample Outputs for each category ---

print("\n--- Sample of Vague Contracts Found: ---")
if not vague_data.empty:
    original_vague_contracts = df[df['CN ID'].isin(vague_data['CN ID'])]
    display_cols = ['Agency', 'Title', 'Value (AUD)', 'Supplier Name']
    actual_display_cols = [col for col in display_cols if col in original_vague_contracts.columns]
    print(original_vague_contracts[actual_display_cols].head(5))
else:
    print("No vague contracts found in this sample.")

print("\n--- Sample of 'Just Under Threshold' Contracts Found: ---")
if not just_under_threshold_data.empty:
    original_threshold_contracts = df[df['CN ID'].isin(just_under_threshold_data['CN ID'])]
    display_cols = ['Agency', 'Title', 'Value (AUD)', 'Supplier Name']
    actual_display_cols = [col for col in display_cols if col in original_threshold_contracts.columns]
    print(original_threshold_contracts[actual_display_cols].head(5))
else:
    print("No 'Just Under Threshold' contracts found in this sample.")

print("\n--- Sample of 'Super Red Flag' Contracts Found: ---")
if not super_red_flag_data.empty:
    original_super_red_flags = df[df['CN ID'].isin(super_red_flag_data['CN ID'])]
    display_cols = ['Agency', 'Title', 'Value (AUD)', 'Supplier Name']
    actual_display_cols = [col for col in display_cols if col in original_super_red_flags.columns]
    print(original_super_red_flags[actual_display_cols].head())
else:
    print("No 'Super Red Flag' contracts found in this sample.")


--- Starting Multi-File Ingestion ---

Processing file: pwc_pace_data.csv...
  Attempted UTF-8 encoding.
✅ Success! Data loaded and headers assigned from pwc_pace_data.csv.
  Rows loaded and cleaned: 155

Processing file: deloitte_pace_data.csv...
  UTF-8 failed, successfully tried Latin-1 encoding.
✅ Success! Data loaded and headers assigned from deloitte_pace_data.csv.
  Rows loaded and cleaned: 3298

Processing file: kmpg_pace_data.csv...
❌ Error: File 'kmpg_pace_data.csv' not found. Please upload it to Colab.

Processing file: ey_pace_data.csv...
  Attempted UTF-8 encoding.
✅ Success! Data loaded and headers assigned from ey_pace_data.csv.
  Rows loaded and cleaned: 72

--- All files concatenated. Initial total rows: 3525 ---
--- Deduplicated 0 rows. Remaining unique contracts: 3525 ---

✅ Final Consolidated Data loaded for analysis.
Total unique contracts: 3525
Columns in final DataFrame: ['CN ID', 'Title', 'Agency', 'Publish Date', 'Category', 'Contract Start Date', 'Execution Da

In [ ]:
# --- PACE MODULE 1 (V12): MULTI-FILE INGESTION, CLEANING, AND ANALYSIS ---

import pandas as pd
import numpy as np # Import numpy for NaN handling

# --- COMMON FUNCTIONS (Defined once) ---

# 1. Function to clean money values (removes '$', ',', converts to float)
def clean_money(value):
    if pd.isna(value):
        return 0.0
    clean_val = str(value).replace('$', '').replace(',', '').strip().replace('(', '-').replace(')', '')
    try:
        return float(clean_val)
    except ValueError:
        return 0.0

# 2. Function to standardize supplier names (specifically for Big 4)
def clean_supplier_name(name):
    if pd.isna(name):
        return "UNKNOWN" # Placeholder for missing names

    name_str = str(name).upper()

    if 'PWC' in name_str or 'PRICEWATERHOUSE' in name_str:
        return 'PwC (Consolidated)'
    elif 'DELOITTE' in name_str:
        return 'Deloitte (Consolidated)'
    elif 'ERNST & YOUNG' in name_str or 'EY' in name_str:
        return 'EY (Consolidated)'
    elif 'KPMG' in name_str:
        return 'KPMG (Consolidated)'
    else:
        return name

# --- MULTI-FILE INGESTION PROCESS ---

# Define the list of files you want to process
file_names = [
    'pwc_pace_data.csv',
    'deloitte_pace_data.csv',
    'kpmg_pace_data.csv',
    'ey_pace_data.csv'
]

all_processed_dfs = [] # This list will hold each cleaned DataFrame

print("--- Starting Multi-File Ingestion ---")

for file_name in file_names:
    print(f"\nProcessing file: {file_name}...")
    try:
        # Load the data: skiprows=16, header=None to handle AusTender's format
        temp_df = pd.read_csv(file_name, encoding='utf-8', skiprows=16, header=None)

        # Manually set headers from the second row (index 1) of the raw loaded data
        temp_df.columns = temp_df.iloc[1]

        # Drop the first two rows (the NaN row and the header row)
        temp_df = temp_df[2:].reset_index(drop=True)

        print(f"✅ Success! Data loaded and headers assigned from {file_name}.")

        # Apply cleaning functions
        temp_df['Clean_Supplier'] = temp_df['Supplier Name'].apply(clean_supplier_name)
        if 'Value (AUD)' in temp_df.columns:
            temp_df['Clean_Value'] = temp_df['Value (AUD)'].apply(clean_money)
        else:
            print(f"❌ 'Value (AUD)' column not found in {file_name}. Assigning 0.0.")
            temp_df['Clean_Value'] = 0.0 # Assign a default to avoid further errors

        all_processed_dfs.append(temp_df)
        print(f"  Rows loaded and cleaned: {len(temp_df)}")

    except FileNotFoundError:
        print(f"❌ Error: File '{file_name}' not found. Please upload it to Colab.")
    except Exception as e:
        print(f"❌ Error processing '{file_name}': {e}")

# --- CONSOLIDATION AND DEDUPLICATION ---

if all_processed_dfs:
    df = pd.concat(all_processed_dfs, ignore_index=True)
    print(f"\n--- All files concatenated. Initial total rows: {len(df)} ---")

    # Remove duplicate contracts based on 'CN ID' (Contract Notice ID)
    # This prevents counting the same contract multiple times if it was found in different searches.
    original_rows = len(df)
    if 'CN ID' in df.columns:
        df.drop_duplicates(subset=['CN ID'], keep='first', inplace=True)
        deduplicated_rows = len(df)
        print(f"--- Deduplicated {original_rows - deduplicated_rows} rows. Remaining unique contracts: {deduplicated_rows} ---")
    else:
        print("⚠️ 'CN ID' column not found for deduplication. Skipping deduplication.")
        deduplicated_rows = original_rows

    print("\n✅ Final Consolidated Data loaded for analysis.")
    print(f"Total unique contracts: {deduplicated_rows}")
    print("Columns in final DataFrame:", list(df.columns))
    print("First 5 rows of consolidated data:")
    print(df.head())

else:
    print("\n❌ No data frames were successfully processed. PACE engine cannot proceed.")
    # Exit here if no data was loaded at all
    raise SystemExit("No data loaded to process.") # Nicer way to exit in Colab than exit()

# --- PACE MODULE 2: CORE ANALYSIS ---

# Ensure Clean_Value is numeric (important for sum operations later)
df['Clean_Value'] = pd.to_numeric(df['Clean_Value'], errors='coerce').fillna(0)

# Analysis A: Total Spend by the Consolidated Big 4 Entities
# Define the Big 4 firms for consolidated reporting
big_4_firms = [
    'PwC (Consolidated)',
    'Deloitte (Consolidated)',
    'EY (Consolidated)',
    'KPMG (Consolidated)'
]

# Analysis B: The "Vague Description" Hunter (now smarter!)
vague_keywords = [
    'management advisory services', 'consulting service', 'strategic advice',
    'professional services', 'advisory services', 'business advisory services',
    'support services', 'review services', 'quality assurance',
    'advisory support', 'program management', 'capacity building',
    'evaluation services', 'transformation services', 'change management'
]

vague_pattern = '|'.join(vague_keywords)
keyword_vague_data = df[
    df['Title'].fillna('').str.lower().str.contains(vague_pattern, na=False)
]

short_titles_data = df[
    df['Title'].fillna('').apply(lambda x: len(str(x).split())) < 4
]

vague_data = pd.concat([keyword_vague_data, short_titles_data]).drop_duplicates(subset=['CN ID'])

total_vague = vague_data['Clean_Value'].sum()
vague_count = len(vague_data)


# Analysis C: The "Just Under Threshold" Hunter
threshold_min = 70000.00
threshold_max = 79999.99

just_under_threshold_data = df[
    (df['Clean_Value'] >= threshold_min) &
    (df['Clean_Value'] <= threshold_max)
]
total_just_under_threshold = just_under_threshold_data['Clean_Value'].sum()
count_just_under_threshold = len(just_under_threshold_data)

# Analysis D: The "Super Red Flag" Hunter (Vague AND Just Under Threshold)
super_red_flag_data = pd.merge(
    vague_data[['CN ID', 'Clean_Value']],
    just_under_threshold_data[['CN ID', 'Clean_Value']],
    on='CN ID',
    how='inner'
)
super_red_flag_count = len(super_red_flag_data)
super_red_flag_total = super_red_flag_data['Clean_Value_x'].sum()


# --- PACE MODULE 3: REPORTING ---
print("\n" + "="*40)
print("      PACE ENGINE - CONSOLIDATED REPORT      ")
print("="*40)

# Report for each of the Big 4 firms
total_big_4_spend = 0
for firm in big_4_firms:
    firm_data = df[df['Clean_Supplier'] == firm]
    firm_spend = firm_data['Clean_Value'].sum()
    total_big_4_spend += firm_spend
    # Check if the firm was actually found in the data before printing
    if firm_spend > 0:
        print(f"💰 Total Spend with {firm:<22}: ${firm_spend:,.2f}")

print(f"\n📈 Grand Total Big 4 Spend (in this data): ${total_big_4_spend:,.2f}")
print("="*40)

print(f"🌫️  Contracts with Vague Descriptions:   {vague_count}")
print(f"💸 Total Value of Vague Contracts:      ${total_vague:,.2f}")
print(f"📉 Contracts Just Under $80k Threshold: {count_just_under_threshold}")
print(f"💵 Value of Just Under Threshold Contracts: ${total_just_under_threshold:,.2f}")
print(f"🚨 Super Red Flag (Vague & Under Threshold): {super_red_flag_count}")
print(f"💲 Value of Super Red Flag Contracts: ${super_red_flag_total:,.2f}")
print("="*40)

# --- Sample Outputs for each category ---

print("\n--- Sample of Vague Contracts Found: ---")
if not vague_data.empty:
    original_vague_contracts = df[df['CN ID'].isin(vague_data['CN ID'])]
    display_cols = ['Agency', 'Title', 'Value (AUD)', 'Supplier Name'] # Added Supplier Name
    actual_display_cols = [col for col in display_cols if col in original_vague_contracts.columns]
    print(original_vague_contracts[actual_display_cols].head(5))
else:
    print("No vague contracts found in this sample.")

print("\n--- Sample of 'Just Under Threshold' Contracts Found: ---")
if not just_under_threshold_data.empty:
    original_threshold_contracts = df[df['CN ID'].isin(just_under_threshold_data['CN ID'])]
    display_cols = ['Agency', 'Title', 'Value (AUD)', 'Supplier Name'] # Added Supplier Name
    actual_display_cols = [col for col in display_cols if col in original_threshold_contracts.columns]
    print(original_threshold_contracts[actual_display_cols].head(5))
else:
    print("No 'Just Under Threshold' contracts found in this sample.")

print("\n--- Sample of 'Super Red Flag' Contracts Found: ---")
if not super_red_flag_data.empty:
    original_super_red_flags = df[df['CN ID'].isin(super_red_flag_data['CN ID'])]
    display_cols = ['Agency', 'Title', 'Value (AUD)', 'Supplier Name'] # Added Supplier Name
    actual_display_cols = [col for col in display_cols if col in original_super_red_flags.columns]
    print(original_super_red_flags[actual_display_cols].head())
else:
    print("No 'Super Red Flag' contracts found in this sample.")


--- Starting Multi-File Ingestion ---

Processing file: pwc_pace_data.csv...
✅ Success! Data loaded and headers assigned from pwc_pace_data.csv.
  Rows loaded and cleaned: 155

Processing file: deloitte_pace_data.csv...
❌ Error processing 'deloitte_pace_data.csv': 'utf-8' codec can't decode byte 0x96 in position 31: invalid start byte

Processing file: kpmg_pace_data.csv...
❌ Error processing 'kpmg_pace_data.csv': 'utf-8' codec can't decode byte 0x92 in position 20: invalid start byte

Processing file: ey_pace_data.csv...
✅ Success! Data loaded and headers assigned from ey_pace_data.csv.
  Rows loaded and cleaned: 72

--- All files concatenated. Initial total rows: 227 ---
--- Deduplicated 0 rows. Remaining unique contracts: 227 ---

✅ Final Consolidated Data loaded for analysis.
Total unique contracts: 227
Columns in final DataFrame: ['CN ID', 'Title', 'Agency', 'Publish Date', 'Category', 'Contract Start Date', 'Execution Date', 'Contract End Date', 'Value (AUD)', 'ATM ID', 'Supplier

In [ ]:
# --- PACE MODULE 1 (V11): DATA INGESTION & CLEANING ---

import pandas as pd
import numpy as np # Import numpy for NaN handling

try:
    # 1. LOAD THE DATA with utf-8 encoding and the determined skiprows=16
    # Use header=None initially to tell pandas not to guess the header.
    df = pd.read_csv('pace_data.csv', encoding='utf-8', skiprows=16, header=None)

    # Debug step: Print what the DataFrame looks like immediately after loading
    print("--- RAW DATA AFTER SKIPPING 16 ROWS (DEBUG) ---")
    print(df.head(3)) # Print first THREE rows to see potential headers and the empty row
    print("--- END RAW DATA DEBUG ---")

    # 2. MANUALLY SET THE HEADERS (using the row at index 1 for headers)
    # The actual headers are in the second row (index 1) of our loaded DataFrame.
    df.columns = df.iloc[1]

    # Then, drop the first two rows (the NaN row at index 0 and the header row at index 1)
    # from the DataFrame, as they are no longer part of the data.
    df = df[2:].reset_index(drop=True)

    print("\n✅ Success! Data loaded and headers correctly assigned.")
    print(f"Total rows found: {len(df)}")
    print("Columns found:", list(df.columns))

    # Let's also print the first 5 rows of the actual data to visually confirm
    print("\nFirst 5 rows of actual data (after header assignment):")
    print(df.head())

except Exception as e:
    print(f"❌ Error loading data: {e}")

# Only proceed if we actually have the data loaded with correct headers
# We're checking for a few key columns this time, just to be sure.
if 'Supplier Name' in df.columns and 'Value (AUD)' in df.columns and 'Title' in df.columns:

    # 2. CLEAN THE SUPPLIER NAMES (The "PwC" Fix)
    def clean_supplier_name(name):
        # Handle NaN values explicitly before converting to upper
        if pd.isna(name):
            return "UNKNOWN" # Or some other placeholder for missing names

        name_str = str(name).upper()
        if 'PWC' in name_str or 'PRICEWATERHOUSE' in name_str:
            return 'PwC (Consolidated)'
        else:
            return name

    df['Clean_Supplier'] = df['Supplier Name'].apply(clean_supplier_name)
    print("✅ Supplier names standardized.")

    # 3. CLEAN THE MONEY COLUMN
    def clean_money(value):
        # Handle cases where value might be NaN (missing)
        if pd.isna(value):
            return 0.0
        # Remove '$', ',', and spaces. Some numbers might be in parentheses for negative.
        clean_val = str(value).replace('$', '').replace(',', '').strip().replace('(', '-').replace(')', '')
        try:
            return float(clean_val)
        except ValueError:
            # If conversion fails, return 0.0 or log an error
            return 0.0

    # Ensure 'Value (AUD)' column exists before trying to clean it
    if 'Value (AUD)' in df.columns:
        df['Clean_Value'] = df['Value (AUD)'].apply(clean_money)
        print("✅ Money values standardized.")
    else:
        print("❌ 'Value (AUD)' column not found after header assignment. Cannot standardize money.")
        df['Clean_Value'] = 0.0 # Assign a default to avoid further errors


    # --- PACE MODULE 2: BASIC ANALYSIS ---

    # Analysis A: Total Spend by the Consolidated PwC Entity
    # Ensure Clean_Value is numeric before summing
    df['Clean_Value'] = pd.to_numeric(df['Clean_Value'], errors='coerce').fillna(0)
    pwc_data = df[df['Clean_Supplier'] == 'PwC (Consolidated)']
    total_pwc = pwc_data['Clean_Value'].sum()

    # Analysis B: The "Vague Description" Hunter (now smarter!)

    # List of keywords that indicate a potentially vague description
    vague_keywords = [
        'management advisory services',
        'consulting service',
        'strategic advice',
        'professional services',
        'advisory services',
        'business advisory services',
        'support services',
        'review services',
        'quality assurance',
        'advisory support', # Added another common one
        'program management' # Another common vague one
    ]

    # 1. Look for specific vague keywords
    # Create a regex pattern to search for any of the vague keywords (case-insensitive)
    vague_pattern = '|'.join(vague_keywords)

    # Filter for titles containing any of the vague keywords
    keyword_vague_data = df[
        df['Title'].fillna('').str.lower().str.contains(vague_pattern, na=False)
    ]

    # 2. Look for extremely short titles (e.g., less than 4 words)
    short_titles_data = df[
        df['Title'].fillna('').apply(lambda x: len(str(x).split())) < 4
    ]

    # Combine both criteria: contracts that match vague keywords OR have very short titles
    # Use .drop_duplicates() to count each unique contract only once
    vague_data = pd.concat([keyword_vague_data, short_titles_data]).drop_duplicates(subset=['CN ID'])

    total_vague = vague_data['Clean_Value'].sum()
    vague_count = len(vague_data)


    # Analysis C: The "Just Under Threshold" Hunter
    # Flagging contracts between $70,000 and $79,999 (inclusive)
    threshold_min = 70000.00
    threshold_max = 79999.99 # To be inclusive up to $79,999

    just_under_threshold_data = df[
        (df['Clean_Value'] >= threshold_min) &
        (df['Clean_Value'] <= threshold_max)
    ]
    total_just_under_threshold = just_under_threshold_data['Clean_Value'].sum()
    count_just_under_threshold = len(just_under_threshold_data)

    # Analysis D: The "Super Red Flag" Hunter (Vague AND Just Under Threshold)
    # Find contracts that appear in BOTH vague_data AND just_under_threshold_data

    # We use pandas .merge() to find the common CN IDs between the two sets
    super_red_flag_data = pd.merge(
        vague_data[['CN ID', 'Clean_Value']],
        just_under_threshold_data[['CN ID', 'Clean_Value']],
        on='CN ID',
        how='inner'
    )

    # The Clean_Value from the merge might be duplicated or from vague_data, so we'll re-calculate from original df
    super_red_flag_count = len(super_red_flag_data)
    super_red_flag_total = super_red_flag_data['Clean_Value_x'].sum() # or Clean_Value_y, they should be the same


    # --- REPORTING ---
    print("\n" + "="*40)
    print("      PACE ENGINE - INITIAL REPORT      ")
    print("="*40)
    print(f"💰 Total Spend with PwC (Consolidated): ${total_pwc:,.2f}")
    print(f"🌫️  Contracts with Vague Descriptions:   {vague_count}")
    print(f"💸 Total Value of Vague Contracts:      ${total_vague:,.2f}")
    print(f"📉 Contracts Just Under $80k Threshold: {count_just_under_threshold}")
    print(f"💵 Value of Just Under Threshold Contracts: ${total_just_under_threshold:,.2f}")
    print(f"🚨 Super Red Flag (Vague & Under Threshold): {super_red_flag_count}")
    print(f"💲 Value of Super Red Flag Contracts: ${super_red_flag_total:,.2f}")
    print("="*40)

    # Optional: Show the first few vague rows to prove it works
    print("\nSample of Vague Contracts Found:")
    if not vague_data.empty:
        display_cols = ['Agency', 'Title', 'Value (AUD)']
        actual_display_cols = [col for col in display_cols if col in vague_data.columns]
        print(vague_data[actual_display_cols].head(5)) # Changed to print top 5
    else:
        print("No vague contracts found in this sample.")

    # Optional: Show a sample of 'Just Under Threshold' contracts
    if not just_under_threshold_data.empty:
        print("\nSample of 'Just Under Threshold' Contracts Found:")
        display_cols = ['Agency', 'Title', 'Value (AUD)']
        actual_display_cols = [col for col in display_cols if col in just_under_threshold_data.columns]
        print(just_under_threshold_data[actual_display_cols].head(5)) # Changed to print top 5
    else:
        print("No 'Just Under Threshold' contracts found in this sample.")

    # Optional: Show a sample of 'Super Red Flag' contracts
    if not super_red_flag_data.empty:
        print("\nSample of 'Super Red Flag' Contracts Found:")
        # We need to get the original data for these CN IDs for full details
        original_super_red_flags = df[df['CN ID'].isin(super_red_flag_data['CN ID'])]
        display_cols_for_super = ['Agency', 'Title', 'Value (AUD)', 'Supplier Name'] # Added Supplier Name
        actual_display_cols_for_super = [col for col in display_cols_for_super if col in original_super_red_flags.columns]
        print(original_super_red_flags[actual_display_cols_for_super].head())
    else:
        print("No 'Super Red Flag' contracts found in this sample.")


else:
    print("\n⚠️ Still didn't find the expected column names. This indicates a persistent issue with CSV structure.")


--- RAW DATA AFTER SKIPPING 16 ROWS (DEBUG) ---
          0                          1   \
0        NaN                        NaN   
1      CN ID                      Title   
2  CN4033077  Data Subscription Service   

                                                  2             3   \
0                                                NaN           NaN   
1                                             Agency  Publish Date   
2  Department of Infrastructure, Transport, Regio...      6-Feb-24   

              4                    5               6                  7   \
0            NaN                  NaN             NaN                NaN   
1       Category  Contract Start Date  Execution Date  Contract End Date   
2  Data services            17-Jan-24             NaN          16-Jan-25   

            8       9                           10                 11  12  
0          NaN     NaN                         NaN                NaN NaN  
1  Value (AUD)  ATM ID               Supp

In [ ]:
# --- PACE MODULE 1 (V10): DATA INGESTION & CLEANING ---

import pandas as pd
import numpy as np # Import numpy for NaN handling

try:
    # 1. LOAD THE DATA with utf-8 encoding and the determined skiprows=16
    # Use header=None initially to tell pandas not to guess the header.
    df = pd.read_csv('pace_data.csv', encoding='utf-8', skiprows=16, header=None)

    # Debug step: Print what the DataFrame looks like immediately after loading
    print("--- RAW DATA AFTER SKIPPING 16 ROWS (DEBUG) ---")
    print(df.head(3)) # Print first THREE rows to see potential headers and the empty row
    print("--- END RAW DATA DEBUG ---")

    # 2. MANUALLY SET THE HEADERS (using the row at index 1 for headers)
    # The actual headers are in the second row (index 1) of our loaded DataFrame.
    df.columns = df.iloc[1]

    # Then, drop the first two rows (the NaN row at index 0 and the header row at index 1)
    # from the DataFrame, as they are no longer part of the data.
    df = df[2:].reset_index(drop=True)

    print("\n✅ Success! Data loaded and headers correctly assigned.")
    print(f"Total rows found: {len(df)}")
    print("Columns found:", list(df.columns))

    # Let's also print the first 5 rows of the actual data to visually confirm
    print("\nFirst 5 rows of actual data (after header assignment):")
    print(df.head())

except Exception as e:
    print(f"❌ Error loading data: {e}")

# Only proceed if we actually have the data loaded with correct headers
# We're checking for a few key columns this time, just to be sure.
if 'Supplier Name' in df.columns and 'Value (AUD)' in df.columns and 'Title' in df.columns:

    # 2. CLEAN THE SUPPLIER NAMES (The "PwC" Fix)
    def clean_supplier_name(name):
        # Handle NaN values explicitly before converting to upper
        if pd.isna(name):
            return "UNKNOWN" # Or some other placeholder for missing names

        name_str = str(name).upper()
        if 'PWC' in name_str or 'PRICEWATERHOUSE' in name_str:
            return 'PwC (Consolidated)'
        else:
            return name

    df['Clean_Supplier'] = df['Supplier Name'].apply(clean_supplier_name)
    print("✅ Supplier names standardized.")

    # 3. CLEAN THE MONEY COLUMN
    def clean_money(value):
        # Handle cases where value might be NaN (missing)
        if pd.isna(value):
            return 0.0
        # Remove '$', ',', and spaces. Some numbers might be in parentheses for negative.
        clean_val = str(value).replace('$', '').replace(',', '').strip().replace('(', '-').replace(')', '')
        try:
            return float(clean_val)
        except ValueError:
            # If conversion fails, return 0.0 or log an error
            return 0.0

    # Ensure 'Value (AUD)' column exists before trying to clean it
    if 'Value (AUD)' in df.columns:
        df['Clean_Value'] = df['Value (AUD)'].apply(clean_money)
        print("✅ Money values standardized.")
    else:
        print("❌ 'Value (AUD)' column not found after header assignment. Cannot standardize money.")
        df['Clean_Value'] = 0.0 # Assign a default to avoid further errors


    # --- PACE MODULE 2: BASIC ANALYSIS ---

    # Analysis A: Total Spend by the Consolidated PwC Entity
    # Ensure Clean_Value is numeric before summing
    df['Clean_Value'] = pd.to_numeric(df['Clean_Value'], errors='coerce').fillna(0)
    pwc_data = df[df['Clean_Supplier'] == 'PwC (Consolidated)']
    total_pwc = pwc_data['Clean_Value'].sum()

    # Analysis B: The "Vague Description" Hunter (now smarter!)

    # List of keywords that indicate a potentially vague description
    vague_keywords = [
        'management advisory services',
        'consulting service',
        'strategic advice',
        'professional services',
        'advisory services',
        'business advisory services',
        'support services',
        'review services', # Added based on your screenshot's "Review of the Department's Reconciliation Action Plan"
        'quality assurance' # Another one from your screenshot "Quality Assurance Framework"
    ]

    # 1. Look for specific vague keywords
    # Create a regex pattern to search for any of the vague keywords (case-insensitive)
    vague_pattern = '|'.join(vague_keywords)

    # Filter for titles containing any of the vague keywords
    keyword_vague_data = df[
        df['Title'].fillna('').str.lower().str.contains(vague_pattern, na=False)
    ]

    # 2. Look for extremely short titles (e.g., less than 4 words)
    short_titles_data = df[
        df['Title'].fillna('').apply(lambda x: len(str(x).split())) < 4
    ]

    # Combine both criteria: contracts that match vague keywords OR have very short titles
    # Use .drop_duplicates() to count each unique contract only once
    vague_data = pd.concat([keyword_vague_data, short_titles_data]).drop_duplicates(subset=['CN ID'])

    total_vague = vague_data['Clean_Value'].sum()
    vague_count = len(vague_data)


    # Analysis C: The "Just Under Threshold" Hunter
    # Flagging contracts between $70,000 and $79,999 (inclusive)
    threshold_min = 70000.00
    threshold_max = 79999.99 # To be inclusive up to $79,999

    just_under_threshold_data = df[
        (df['Clean_Value'] >= threshold_min) &
        (df['Clean_Value'] <= threshold_max)
    ]
    total_just_under_threshold = just_under_threshold_data['Clean_Value'].sum()
    count_just_under_threshold = len(just_under_threshold_data)

    # --- REPORTING ---
    print("\n" + "="*40)
    print("      PACE ENGINE - INITIAL REPORT      ")
    print("="*40)
    print(f"💰 Total Spend with PwC (Consolidated): ${total_pwc:,.2f}")
    print(f"🌫️  Contracts with Vague Descriptions:   {vague_count}")
    print(f"💸 Total Value of Vague Contracts:      ${total_vague:,.2f}")
    print(f"📉 Contracts Just Under $80k Threshold: {count_just_under_threshold}")
    print(f"💵 Value of Just Under Threshold Contracts: ${total_just_under_threshold:,.2f}")
    print("="*40)

    # Optional: Show the first few vague rows to prove it works
    print("\nSample of Vague Contracts Found:")
    if not vague_data.empty:
        display_cols = ['Agency', 'Title', 'Value (AUD)']
        actual_display_cols = [col for col in display_cols if col in vague_data.columns]
        print(vague_data[actual_display_cols].head(5)) # Changed to print top 5
    else:
        print("No vague contracts found in this sample.")

    # Optional: Show a sample of 'Just Under Threshold' contracts
    if not just_under_threshold_data.empty:
        print("\nSample of 'Just Under Threshold' Contracts Found:")
        display_cols = ['Agency', 'Title', 'Value (AUD)']
        actual_display_cols = [col for col in display_cols if col in just_under_threshold_data.columns]
        print(just_under_threshold_data[actual_display_cols].head(5)) # Changed to print top 5
    else:
        print("No 'Just Under Threshold' contracts found in this sample.")


else:
    print("\n⚠️ Still didn't find the expected column names. This indicates a persistent issue with CSV structure.")


--- RAW DATA AFTER SKIPPING 16 ROWS (DEBUG) ---
          0                          1   \
0        NaN                        NaN   
1      CN ID                      Title   
2  CN4033077  Data Subscription Service   

                                                  2             3   \
0                                                NaN           NaN   
1                                             Agency  Publish Date   
2  Department of Infrastructure, Transport, Regio...      6-Feb-24   

              4                    5               6                  7   \
0            NaN                  NaN             NaN                NaN   
1       Category  Contract Start Date  Execution Date  Contract End Date   
2  Data services            17-Jan-24             NaN          16-Jan-25   

            8       9                           10                 11  12  
0          NaN     NaN                         NaN                NaN NaN  
1  Value (AUD)  ATM ID               Supp

In [ ]:
# --- PACE MODULE 1 (V9): DATA INGESTION & CLEANING ---

import pandas as pd
import numpy as np # Import numpy for NaN handling

try:
    # 1. LOAD THE DATA with utf-8 encoding and the determined skiprows=16
    # Use header=None initially to tell pandas not to guess the header.
    df = pd.read_csv('pace_data.csv', encoding='utf-8', skiprows=16, header=None)

    # Debug step: Print what the DataFrame looks like immediately after loading
    print("--- RAW DATA AFTER SKIPPING 16 ROWS (DEBUG) ---")
    print(df.head(3)) # Print first THREE rows to see potential headers and the empty row
    print("--- END RAW DATA DEBUG ---")

    # 2. MANUALLY SET THE HEADERS (using the row at index 1 for headers)
    # The actual headers are in the second row (index 1) of our loaded DataFrame.
    df.columns = df.iloc[1]

    # Then, drop the first two rows (the NaN row at index 0 and the header row at index 1)
    # from the DataFrame, as they are no longer part of the data.
    df = df[2:].reset_index(drop=True)

    print("\n✅ Success! Data loaded and headers correctly assigned.")
    print(f"Total rows found: {len(df)}")
    print("Columns found:", list(df.columns))

    # Let's also print the first 5 rows of the actual data to visually confirm
    print("\nFirst 5 rows of actual data (after header assignment):")
    print(df.head())

except Exception as e:
    print(f"❌ Error loading data: {e}")

# Only proceed if we actually have the data loaded with correct headers
# We're checking for a few key columns this time, just to be sure.
if 'Supplier Name' in df.columns and 'Value (AUD)' in df.columns and 'Title' in df.columns:

    # 2. CLEAN THE SUPPLIER NAMES (The "PwC" Fix)
    def clean_supplier_name(name):
        # Handle NaN values explicitly before converting to upper
        if pd.isna(name):
            return "UNKNOWN" # Or some other placeholder for missing names

        name_str = str(name).upper()
        if 'PWC' in name_str or 'PRICEWATERHOUSE' in name_str:
            return 'PwC (Consolidated)'
        else:
            return name

    df['Clean_Supplier'] = df['Supplier Name'].apply(clean_supplier_name)
    print("✅ Supplier names standardized.")

    # 3. CLEAN THE MONEY COLUMN
    def clean_money(value):
        # Handle cases where value might be NaN (missing)
        if pd.isna(value):
            return 0.0
        # Remove '$', ',', and spaces. Some numbers might be in parentheses for negative.
        clean_val = str(value).replace('$', '').replace(',', '').strip().replace('(', '-').replace(')', '')
        try:
            return float(clean_val)
        except ValueError:
            # If conversion fails, return 0.0 or log an error
            return 0.0

    # Ensure 'Value (AUD)' column exists before trying to clean it
    if 'Value (AUD)' in df.columns:
        df['Clean_Value'] = df['Value (AUD)'].apply(clean_money)
        print("✅ Money values standardized.")
    else:
        print("❌ 'Value (AUD)' column not found after header assignment. Cannot standardize money.")
        df['Clean_Value'] = 0.0 # Assign a default to avoid further errors


    # --- PACE MODULE 2: BASIC ANALYSIS ---

    # Analysis A: Total Spend by the Consolidated PwC Entity
    # Ensure Clean_Value is numeric before summing
    df['Clean_Value'] = pd.to_numeric(df['Clean_Value'], errors='coerce').fillna(0)
    pwc_data = df[df['Clean_Supplier'] == 'PwC (Consolidated)']
    total_pwc = pwc_data['Clean_Value'].sum()

    # Analysis B: The "Vague Description" Hunter
    # Looking for titles exactly matching "Management advisory services"
    # Added .fillna('') to handle potential NaN values in 'Title'
    vague_data = df[df['Title'].fillna('').str.lower().str.strip() == 'management advisory services']
    total_vague = vague_data['Clean_Value'].sum()
    vague_count = len(vague_data)

    # Analysis C: The "Just Under Threshold" Hunter
    # Flagging contracts between $70,000 and $79,999 (inclusive)
    threshold_min = 70000.00
    threshold_max = 79999.99 # To be inclusive up to $79,999

    just_under_threshold_data = df[
        (df['Clean_Value'] >= threshold_min) &
        (df['Clean_Value'] <= threshold_max)
    ]
    total_just_under_threshold = just_under_threshold_data['Clean_Value'].sum()
    count_just_under_threshold = len(just_under_threshold_data)

    # --- REPORTING ---
    print("\n" + "="*40)
    print("      PACE ENGINE - INITIAL REPORT      ")
    print("="*40)
    print(f"💰 Total Spend with PwC (Consolidated): ${total_pwc:,.2f}")
    print(f"🌫️  Contracts with Vague Descriptions:   {vague_count}")
    print(f"💸 Total Value of Vague Contracts:      ${total_vague:,.2f}")
    print(f"📉 Contracts Just Under $80k Threshold: {count_just_under_threshold}")
    print(f"💵 Value of Just Under Threshold Contracts: ${total_just_under_threshold:,.2f}")
    print("="*40)

    # Optional: Show the first few vague rows to prove it works
    print("\nSample of Vague Contracts Found:")
    if not vague_data.empty:
        display_cols = ['Title', 'Value (AUD)']
        if 'Agency' in vague_data.columns:
            display_cols.insert(0, 'Agency')
        print(vague_data[display_cols].head())
    else:
        print("No vague contracts found in this sample.")

    # Optional: Show a sample of 'Just Under Threshold' contracts
    if not just_under_threshold_data.empty:
        print("\nSample of 'Just Under Threshold' Contracts Found:")
        display_cols = ['Agency', 'Title', 'Value (AUD)']
        actual_display_cols = [col for col in display_cols if col in just_under_threshold_data.columns]
        print(just_under_threshold_data[actual_display_cols].head())
    else:
        print("No 'Just Under Threshold' contracts found in this sample.")


else:
    print("\n⚠️ Still didn't find the expected column names. This indicates a persistent issue with CSV structure.")


--- RAW DATA AFTER SKIPPING 16 ROWS (DEBUG) ---
          0                          1   \
0        NaN                        NaN   
1      CN ID                      Title   
2  CN4033077  Data Subscription Service   

                                                  2             3   \
0                                                NaN           NaN   
1                                             Agency  Publish Date   
2  Department of Infrastructure, Transport, Regio...      6-Feb-24   

              4                    5               6                  7   \
0            NaN                  NaN             NaN                NaN   
1       Category  Contract Start Date  Execution Date  Contract End Date   
2  Data services            17-Jan-24             NaN          16-Jan-25   

            8       9                           10                 11  12  
0          NaN     NaN                         NaN                NaN NaN  
1  Value (AUD)  ATM ID               Supp

In [ ]:
# --- PACE MODULE 1 (V8): DATA INGESTION & CLEANING ---

import pandas as pd
import numpy as np # Import numpy for NaN handling

try:
    # 1. LOAD THE DATA with utf-8 encoding and the determined skiprows=16
    # Use header=None initially to tell pandas not to guess the header.
    df = pd.read_csv('pace_data.csv', encoding='utf-8', skiprows=16, header=None)

    # Debug step: Print what the DataFrame looks like immediately after loading
    print("--- RAW DATA AFTER SKIPPING 16 ROWS (DEBUG) ---")
    print(df.head(3)) # Print first THREE rows to see potential headers and the empty row
    print("--- END RAW DATA DEBUG ---")

    # 2. MANUALLY SET THE HEADERS (using the row at index 1 for headers)
    # The actual headers are in the second row (index 1) of our loaded DataFrame.
    df.columns = df.iloc[1]

    # Then, drop the first two rows (the NaN row at index 0 and the header row at index 1)
    # from the DataFrame, as they are no longer part of the data.
    df = df[2:].reset_index(drop=True)

    print("\n✅ Success! Data loaded and headers correctly assigned.")
    print(f"Total rows found: {len(df)}")
    print("Columns found:", list(df.columns))

    # Let's also print the first 5 rows of the actual data to visually confirm
    print("\nFirst 5 rows of actual data (after header assignment):")
    print(df.head())

except Exception as e:
    print(f"❌ Error loading data: {e}")

# Only proceed if we actually have the data loaded with correct headers
# We're checking for a few key columns this time, just to be sure.
if 'Supplier Name' in df.columns and 'Value (AUD)' in df.columns and 'Title' in df.columns:

    # 2. CLEAN THE SUPPLIER NAMES (The "PwC" Fix)
    def clean_supplier_name(name):
        # Handle NaN values explicitly before converting to upper
        if pd.isna(name):
            return "UNKNOWN" # Or some other placeholder for missing names

        name_str = str(name).upper()
        if 'PWC' in name_str or 'PRICEWATERHOUSE' in name_str:
            return 'PwC (Consolidated)'
        else:
            return name

    df['Clean_Supplier'] = df['Supplier Name'].apply(clean_supplier_name)
    print("✅ Supplier names standardized.")

    # 3. CLEAN THE MONEY COLUMN
    def clean_money(value):
        # Handle cases where value might be NaN (missing)
        if pd.isna(value):
            return 0.0
        # Remove '$', ',', and spaces. Some numbers might be in parentheses for negative.
        clean_val = str(value).replace('$', '').replace(',', '').strip().replace('(', '-').replace(')', '')
        try:
            return float(clean_val)
        except ValueError:
            # If conversion fails, return 0.0 or log an error
            return 0.0

    # Ensure 'Value (AUD)' column exists before trying to clean it
    if 'Value (AUD)' in df.columns:
        df['Clean_Value'] = df['Value (AUD)'].apply(clean_money)
        print("✅ Money values standardized.")
    else:
        print("❌ 'Value (AUD)' column not found after header assignment. Cannot standardize money.")
        df['Clean_Value'] = 0.0 # Assign a default to avoid further errors


    # --- PACE MODULE 2: BASIC ANALYSIS ---

    # Analysis A: Total Spend by the Consolidated PwC Entity
    # Ensure Clean_Value is numeric before summing
    df['Clean_Value'] = pd.to_numeric(df['Clean_Value'], errors='coerce').fillna(0)
    pwc_data = df[df['Clean_Supplier'] == 'PwC (Consolidated)']
    total_pwc = pwc_data['Clean_Value'].sum()

    # Analysis B: The "Vague Description" Hunter
    # Looking for titles exactly matching "Management advisory services"
    # Added .fillna('') to handle potential NaN values in 'Title'
    vague_data = df[df['Title'].fillna('').str.lower().str.strip() == 'management advisory services']
    total_vague = vague_data['Clean_Value'].sum()
    vague_count = len(vague_data)

    # --- REPORTING ---
    print("\n" + "="*40)
    print("      PACE ENGINE - INITIAL REPORT      ")
    print("="*40)
    print(f"💰 Total Spend with PwC (Consolidated): ${total_pwc:,.2f}")
    print(f"🌫️  Contracts with Vague Descriptions:   {vague_count}")
    print(f"💸 Total Value of Vague Contracts:      ${total_vague:,.2f}")
    print("="*40)

    # Optional: Show the first few vague rows to prove it works
    print("\nSample of Vague Contracts Found:")
    # Only show if there are actual vague contracts
    if not vague_data.empty:
        # Check if 'Agency' column exists before trying to display it
        display_cols = ['Title', 'Value (AUD)']
        if 'Agency' in vague_data.columns:
            display_cols.insert(0, 'Agency')
        print(vague_data[display_cols].head())
    else:
        print("No vague contracts found in this sample.")

else:
    print("\n⚠️ Still didn't find the expected column names. This indicates a persistent issue with CSV structure.")


--- RAW DATA AFTER SKIPPING 16 ROWS (DEBUG) ---
             0                                                  1   \
0           NaN                                                NaN   
1         CN ID                                              Title   
2  CN3820600-A2  Project Assurance Services for the Schools Uni...   

                        2             3                             4   \
0                      NaN           NaN                           NaN   
1                   Agency  Publish Date                      Category   
2  Department of Education     15-Oct-21  Management advisory services   

                    5               6                  7             8   \
0                  NaN             NaN                NaN           NaN   
1  Contract Start Date  Execution Date  Contract End Date   Value (AUD)   
2             1-Oct-21             NaN           2-Oct-23  2,704,622.80   

           9              10                 11  12  
0         NaN      

In [ ]:
# --- PACE MODULE 1 (V7): DATA INGESTION & CLEANING ---

import pandas as pd
import numpy as np # Import numpy for NaN handling

try:
    # 1. LOAD THE DATA with utf-8 encoding and the determined skiprows=16
    # We still use header=None initially to avoid pandas making assumptions.
    df = pd.read_csv('pace_data.csv', encoding='utf-8', skiprows=16, header=None)

    # Debug step: Print what the DataFrame looks like immediately after loading
    print("--- RAW DATA AFTER SKIPPING 16 ROWS (DEBUG) ---")
    print(df.head(2)) # Print first two rows to see potential headers
    print("--- END RAW DATA DEBUG ---")

    # 2. MANUALLY SET THE HEADERS (assuming they are in the first row after skipping)
    # The actual headers are in the first row (index 0) of our loaded DataFrame.
    # We set these as the new column names.
    df.columns = df.iloc[0]

    # Then, we drop that row from the DataFrame, as it's now our header.
    df = df[1:].reset_index(drop=True)

    print("\n✅ Success! Data loaded and headers correctly assigned.")
    print(f"Total rows found: {len(df)}")
    print("Columns found:", list(df.columns))

    # Let's also print the first 5 rows of the actual data to visually confirm
    print("\nFirst 5 rows of actual data (after header assignment):")
    print(df.head())

except Exception as e:
    print(f"❌ Error loading data: {e}")

# Only proceed if we actually have the data loaded with correct headers
# We're checking for a few key columns this time, just to be sure.
if 'Supplier Name' in df.columns and 'Value (AUD)' in df.columns and 'Title' in df.columns:

    # 2. CLEAN THE SUPPLIER NAMES (The "PwC" Fix)
    def clean_supplier_name(name):
        # Handle NaN values explicitly before converting to upper
        if pd.isna(name):
            return "UNKNOWN" # Or some other placeholder for missing names

        name_str = str(name).upper()
        if 'PWC' in name_str or 'PRICEWATERHOUSE' in name_str:
            return 'PwC (Consolidated)'
        else:
            return name

    df['Clean_Supplier'] = df['Supplier Name'].apply(clean_supplier_name)
    print("✅ Supplier names standardized.")

    # 3. CLEAN THE MONEY COLUMN
    def clean_money(value):
        # Handle cases where value might be NaN (missing)
        if pd.isna(value):
            return 0.0
        # Remove '$', ',', and spaces. Some numbers might be in parentheses for negative.
        clean_val = str(value).replace('$', '').replace(',', '').strip().replace('(', '-').replace(')', '')
        try:
            return float(clean_val)
        except ValueError:
            # If conversion fails, return 0.0 or log an error
            return 0.0

    # Ensure 'Value (AUD)' column exists before trying to clean it
    if 'Value (AUD)' in df.columns:
        df['Clean_Value'] = df['Value (AUD)'].apply(clean_money)
        print("✅ Money values standardized.")
    else:
        print("❌ 'Value (AUD)' column not found after header assignment. Cannot standardize money.")
        df['Clean_Value'] = 0.0 # Assign a default to avoid further errors


    # --- PACE MODULE 2: BASIC ANALYSIS ---

    # Analysis A: Total Spend by the Consolidated PwC Entity
    # Ensure Clean_Value is numeric before summing
    df['Clean_Value'] = pd.to_numeric(df['Clean_Value'], errors='coerce').fillna(0)
    pwc_data = df[df['Clean_Supplier'] == 'PwC (Consolidated)']
    total_pwc = pwc_data['Clean_Value'].sum()

    # Analysis B: The "Vague Description" Hunter
    # Looking for titles exactly matching "Management advisory services"
    # Added .fillna('') to handle potential NaN values in 'Title'
    vague_data = df[df['Title'].fillna('').str.lower().str.strip() == 'management advisory services']
    total_vague = vague_data['Clean_Value'].sum()
    vague_count = len(vague_data)

    # --- REPORTING ---
    print("\n" + "="*40)
    print("      PACE ENGINE - INITIAL REPORT      ")
    print("="*40)
    print(f"💰 Total Spend with PwC (Consolidated): ${total_pwc:,.2f}")
    print(f"🌫️  Contracts with Vague Descriptions:   {vague_count}")
    print(f"💸 Total Value of Vague Contracts:      ${total_vague:,.2f}")
    print("="*40)

    # Optional: Show the first few vague rows to prove it works
    print("\nSample of Vague Contracts Found:")
    # Only show if there are actual vague contracts
    if not vague_data.empty:
        # Check if 'Agency' column exists before trying to display it
        display_cols = ['Title', 'Value (AUD)']
        if 'Agency' in vague_data.columns:
            display_cols.insert(0, 'Agency')
        print(vague_data[display_cols].head())
    else:
        print("No vague contracts found in this sample.")

else:
    print("\n⚠️ Still didn't find the expected column names. This indicates a persistent issue with CSV structure.")


--- RAW DATA AFTER SKIPPING 16 ROWS (DEBUG) ---
      0      1       2             3         4                    5   \
0    NaN    NaN     NaN           NaN       NaN                  NaN   
1  CN ID  Title  Agency  Publish Date  Category  Contract Start Date   

               6                  7            8       9              10  \
0             NaN                NaN          NaN     NaN            NaN   
1  Execution Date  Contract End Date  Value (AUD)  ATM ID  Supplier Name   

             11  12  
0           NaN NaN  
1  Last Updated NaN  
--- END RAW DATA DEBUG ---

✅ Success! Data loaded and headers correctly assigned.
Total rows found: 49
Columns found: [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, np.float64(nan)]

First 5 rows of actual data (after header assignment):
0           NaN                                                NaN  \
0         CN ID                                              Title   
1  CN3820600-A2  Project Assurance Services for

In [ ]:
# --- PACE MODULE 1 (V6): DATA INGESTION & CLEANING ---

import pandas as pd
import numpy as np # Import numpy for NaN handling

try:
    # 1. LOAD THE DATA WITHOUT ASSUMING A HEADER INITIALLY
    # We skip 16 rows, but tell pandas there's no header yet (header=None).
    # This makes the "CN ID, Title, Agency..." row the first row of data (index 0).
    df = pd.read_csv('pace_data.csv', encoding='ISO-8559-1', skiprows=16, header=None)

    # 2. MANUALLY SET THE HEADERS
    # The actual headers are in the first row (index 0) of our loaded DataFrame.
    # We set these as the new column names.
    df.columns = df.iloc[0]

    # Then, we drop that row from the DataFrame, as it's now our header.
    df = df[1:].reset_index(drop=True)

    print("✅ Success! Data loaded and headers correctly assigned.")
    print(f"Total rows found: {len(df)}")
    print("Columns found:", list(df.columns))

    # Let's also print the first 5 rows of the actual data to visually confirm
    print("\nFirst 5 rows of actual data (including headers):")
    print(df.head())

except Exception as e:
    print(f"❌ Error loading data: {e}")

# Only proceed if we actually have the data loaded with correct headers
# We're checking for a few key columns this time, just to be sure.
if 'Supplier Name' in df.columns and 'Value (AUD)' in df.columns and 'Title' in df.columns:

    # 2. CLEAN THE SUPPLIER NAMES (The "PwC" Fix)
    def clean_supplier_name(name):
        # Handle NaN values explicitly before converting to upper
        if pd.isna(name):
            return "UNKNOWN" # Or some other placeholder for missing names

        name_str = str(name).upper()
        if 'PWC' in name_str or 'PRICEWATERHOUSE' in name_str:
            return 'PwC (Consolidated)'
        else:
            return name

    df['Clean_Supplier'] = df['Supplier Name'].apply(clean_supplier_name)
    print("✅ Supplier names standardized.")

    # 3. CLEAN THE MONEY COLUMN
    def clean_money(value):
        # Handle cases where value might be NaN (missing)
        if pd.isna(value):
            return 0.0
        # Remove '$', ',', and spaces. Some numbers might be in parentheses for negative.
        clean_val = str(value).replace('$', '').replace(',', '').strip().replace('(', '-').replace(')', '')
        try:
            return float(clean_val)
        except ValueError:
            # If conversion fails, return 0.0 or log an error
            return 0.0

    # Ensure 'Value (AUD)' column exists before trying to clean it
    if 'Value (AUD)' in df.columns:
        df['Clean_Value'] = df['Value (AUD)'].apply(clean_money)
        print("✅ Money values standardized.")
    else:
        print("❌ 'Value (AUD)' column not found after header assignment. Cannot standardize money.")
        df['Clean_Value'] = 0.0 # Assign a default to avoid further errors


    # --- PACE MODULE 2: BASIC ANALYSIS ---

    # Analysis A: Total Spend by the Consolidated PwC Entity
    # Ensure Clean_Value is numeric before summing
    df['Clean_Value'] = pd.to_numeric(df['Clean_Value'], errors='coerce').fillna(0)
    pwc_data = df[df['Clean_Supplier'] == 'PwC (Consolidated)']
    total_pwc = pwc_data['Clean_Value'].sum()

    # Analysis B: The "Vague Description" Hunter
    # Looking for titles exactly matching "Management advisory services"
    # Added .fillna('') to handle potential NaN values in 'Title'
    vague_data = df[df['Title'].fillna('').str.lower().str.strip() == 'management advisory services']
    total_vague = vague_data['Clean_Value'].sum()
    vague_count = len(vague_data)

    # --- REPORTING ---
    print("\n" + "="*40)
    print("      PACE ENGINE - INITIAL REPORT      ")
    print("="*40)
    print(f"💰 Total Spend with PwC (Consolidated): ${total_pwc:,.2f}")
    print(f"🌫️  Contracts with Vague Descriptions:   {vague_count}")
    print(f"💸 Total Value of Vague Contracts:      ${total_vague:,.2f}")
    print("="*40)

    # Optional: Show the first few vague rows to prove it works
    print("\nSample of Vague Contracts Found:")
    # Only show if there are actual vague contracts
    if not vague_data.empty:
        # Check if 'Agency' column exists before trying to display it
        display_cols = ['Title', 'Value (AUD)']
        if 'Agency' in vague_data.columns:
            display_cols.insert(0, 'Agency')
        print(vague_data[display_cols].head())
    else:
        print("No vague contracts found in this sample.")

else:
    print("\n⚠️ Still didn't find the expected column names. This indicates a persistent issue with CSV structure.")


❌ Error loading data: unknown encoding: ISO-8559-1

⚠️ Still didn't find the expected column names. This indicates a persistent issue with CSV structure.


In [ ]:
# --- PACE MODULE 1 (V5): DATA INGESTION & CLEANING ---

import pandas as pd
import numpy as np # Import numpy for NaN handling

# 1. LOAD THE DATA with the FINAL precise SKIPPING
# Based on the latest output, the actual headers are on physical row 17 of the CSV,
# so we need to skip the 16 rows before it (0-indexed).
try:
    df = pd.read_csv('pace_data.csv', encoding='ISO-8859-1', skiprows=16)

    # We'll print the columns again to be absolutely sure
    print("✅ Success! Data loaded correctly.")
    print(f"Total rows found: {len(df)}")
    print("Columns found:", list(df.columns))

    # Let's also print the first 5 rows of the actual data to visually confirm
    print("\nFirst 5 rows of actual data (including headers):")
    print(df.head())

except Exception as e:
    print(f"❌ Error loading data: {e}")

# Only proceed if we actually have the data loaded with correct headers
# We're checking for a few key columns this time, just to be sure.
if 'Supplier Name' in df.columns and 'Value (AUD)' in df.columns and 'Title' in df.columns:

    # 2. CLEAN THE SUPPLIER NAMES (The "PwC" Fix)
    def clean_supplier_name(name):
        # Handle NaN values explicitly before converting to upper
        if pd.isna(name):
            return "UNKNOWN" # Or some other placeholder

        name_str = str(name).upper()
        if 'PWC' in name_str or 'PRICEWATERHOUSE' in name_str:
            return 'PwC (Consolidated)'
        else:
            return name

    df['Clean_Supplier'] = df['Supplier Name'].apply(clean_supplier_name)
    print("✅ Supplier names standardized.")

    # 3. CLEAN THE MONEY COLUMN
    def clean_money(value):
        # Handle cases where value might be NaN (missing)
        if pd.isna(value):
            return 0.0
        # Remove '$', ',', and spaces. Some numbers might be in parentheses for negative.
        clean_val = str(value).replace('$', '').replace(',', '').strip().replace('(', '-').replace(')', '')
        try:
            return float(clean_val)
        except ValueError:
            # If conversion fails, return 0.0 or log an error
            return 0.0

    df['Clean_Value'] = df['Value (AUD)'].apply(clean_money)
    print("✅ Money values standardized.")

    # --- PACE MODULE 2: BASIC ANALYSIS ---

    # Analysis A: Total Spend by the Consolidated PwC Entity
    pwc_data = df[df['Clean_Supplier'] == 'PwC (Consolidated)']
    total_pwc = pwc_data['Clean_Value'].sum()

    # Analysis B: The "Vague Description" Hunter
    # Looking for titles exactly matching "Management advisory services"
    # Added .fillna('') to handle potential NaN values in 'Title'
    vague_data = df[df['Title'].fillna('').str.lower() == 'management advisory services']
    total_vague = vague_data['Clean_Value'].sum()
    vague_count = len(vague_data)

    # --- REPORTING ---
    print("\n" + "="*40)
    print("      PACE ENGINE - INITIAL REPORT      ")
    print("="*40)
    print(f"💰 Total Spend with PwC (Consolidated): ${total_pwc:,.2f}")
    print(f"🌫️  Contracts with Vague Descriptions:   {vague_count}")
    print(f"💸 Total Value of Vague Contracts:      ${total_vague:,.2f}")
    print("="*40)

    # Optional: Show the first few vague rows to prove it works
    print("\nSample of Vague Contracts Found:")
    # Only show if there are actual vague contracts
    if not vague_data.empty:
        # Check if 'Agency' column exists before trying to display it
        display_cols = ['Title', 'Value (AUD)']
        if 'Agency' in vague_data.columns:
            display_cols.insert(0, 'Agency')
        print(vague_data[display_cols].head())
    else:
        print("No vague contracts found in this sample.")

else:
    print("\n⚠️ Still didn't find the expected column names. Adjust 'skiprows' or check your CSV file again.")


✅ Success! Data loaded correctly.
Total rows found: 49
Columns found: ['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12']

First 5 rows of actual data (including headers):
     Unnamed: 0                                         Unnamed: 1  \
0         CN ID                                              Title   
1  CN3820600-A2  Project Assurance Services for the Schools Uni...   
2     CN3922991                       Management advisory services   
3     CN3903240  Payroll voluntary redundancy advisory services...   
4     CN3900356              Supplier Accreditation System Support   

                                          Unnamed: 2    Unnamed: 3  \
0                                             Agency  Publish Date   
1                            Department of Education     15-Oct-21   
2  Office of the Official Secretary to the Govern...      8-Nov

In [ ]:
# --- PACE MODULE 1 (V4): DATA INGESTION & CLEANING ---

import pandas as pd
import numpy as np # Import numpy for NaN handling

# 1. LOAD THE DATA with a more precise SKIPPING
# Based on the latest output, the actual headers are on what would be row 16 of the CSV
# (meaning we need to skip 15 rows before it).
try:
    df = pd.read_csv('pace_data.csv', encoding='ISO-8859-1', skiprows=15)

    # We'll print the columns again to be absolutely sure
    print("✅ Success! Data loaded correctly.")
    print(f"Total rows found: {len(df)}")
    print("Columns found:", list(df.columns))

    # Let's also print the first 5 rows of the actual data to visually confirm
    print("\nFirst 5 rows of actual data (including headers):")
    print(df.head())

except Exception as e:
    print(f"❌ Error loading data: {e}")

# Only proceed if we actually have the data loaded with correct headers
# We're checking for a few key columns this time, just to be sure.
if 'Supplier Name' in df.columns and 'Value (AUD)' in df.columns and 'Title' in df.columns:

    # 2. CLEAN THE SUPPLIER NAMES (The "PwC" Fix)
    def clean_supplier_name(name):
        # Handle NaN values explicitly before converting to upper
        if pd.isna(name):
            return "UNKNOWN" # Or some other placeholder

        name_str = str(name).upper()
        if 'PWC' in name_str or 'PRICEWATERHOUSE' in name_str:
            return 'PwC (Consolidated)'
        else:
            return name

    df['Clean_Supplier'] = df['Supplier Name'].apply(clean_supplier_name)
    print("✅ Supplier names standardized.")

    # 3. CLEAN THE MONEY COLUMN
    def clean_money(value):
        # Handle cases where value might be NaN (missing)
        if pd.isna(value):
            return 0.0
        # Remove '$', ',', and spaces. Some numbers might be in parentheses for negative.
        clean_val = str(value).replace('$', '').replace(',', '').strip().replace('(', '-').replace(')', '')
        try:
            return float(clean_val)
        except ValueError:
            # If conversion fails, return 0.0 or log an error
            return 0.0

    df['Clean_Value'] = df['Value (AUD)'].apply(clean_money)
    print("✅ Money values standardized.")

    # --- PACE MODULE 2: BASIC ANALYSIS ---

    # Analysis A: Total Spend by the Consolidated PwC Entity
    pwc_data = df[df['Clean_Supplier'] == 'PwC (Consolidated)']
    total_pwc = pwc_data['Clean_Value'].sum()

    # Analysis B: The "Vague Description" Hunter
    # Looking for titles exactly matching "Management advisory services"
    # Added .fillna('') to handle potential NaN values in 'Title'
    vague_data = df[df['Title'].fillna('').str.lower() == 'management advisory services']
    total_vague = vague_data['Clean_Value'].sum()
    vague_count = len(vague_data)

    # --- REPORTING ---
    print("\n" + "="*40)
    print("      PACE ENGINE - INITIAL REPORT      ")
    print("="*40)
    print(f"💰 Total Spend with PwC (Consolidated): ${total_pwc:,.2f}")
    print(f"🌫️  Contracts with Vague Descriptions:   {vague_count}")
    print(f"💸 Total Value of Vague Contracts:      ${total_vague:,.2f}")
    print("="*40)

    # Optional: Show the first few vague rows to prove it works
    print("\nSample of Vague Contracts Found:")
    # Only show if there are actual vague contracts
    if not vague_data.empty:
        # Check if 'Agency' column exists before trying to display it
        display_cols = ['Title', 'Value (AUD)']
        if 'Agency' in vague_data.columns:
            display_cols.insert(0, 'Agency')
        print(vague_data[display_cols].head())
    else:
        print("No vague contracts found in this sample.")

else:
    print("\n⚠️ Still didn't find the expected column names. Adjust 'skiprows' or check your CSV file again.")


✅ Success! Data loaded correctly.
Total rows found: 50
Columns found: ['Consultancy?', 'All', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12']

First 5 rows of actual data (including headers):
   Consultancy?                                                All  \
0           NaN                                                NaN   
1         CN ID                                              Title   
2  CN3820600-A2  Project Assurance Services for the Schools Uni...   
3     CN3922991                       Management advisory services   
4     CN3903240  Payroll voluntary redundancy advisory services...   

                                          Unnamed: 2    Unnamed: 3  \
0                                                NaN           NaN   
1                                             Agency  Publish Date   
2                            Department of Education     15-Oct-21  

In [ ]:
# --- PACE MODULE 1 (V3): DATA INGESTION & CLEANING ---

import pandas as pd

# 1. LOAD THE DATA with a more precise SKIPPING
# Based on the screenshot, the actual headers are on row 17 (so skip 16 rows)
try:
    df = pd.read_csv('pace_data.csv', encoding='ISO-8859-1', skiprows=16)

    # We'll print the columns again to be absolutely sure
    print("✅ Success! Data loaded correctly.")
    print(f"Total rows found: {len(df)}")
    print("Columns found:", list(df.columns))

    # Let's also print the first 5 rows of the actual data to visually confirm
    print("\nFirst 5 rows of actual data (including headers):")
    print(df.head())

except Exception as e:
    print(f"❌ Error loading data: {e}")

# Only proceed if we actually have the data loaded with correct headers
# We're checking for a few key columns this time, just to be sure.
if 'Supplier Name' in df.columns and 'Value (AUD)' in df.columns and 'Title' in df.columns:

    # 2. CLEAN THE SUPPLIER NAMES (The "PwC" Fix)
    def clean_supplier_name(name):
        name_str = str(name).upper()
        if 'PWC' in name_str or 'PRICEWATERHOUSE' in name_str:
            return 'PwC (Consolidated)'
        else:
            return name

    df['Clean_Supplier'] = df['Supplier Name'].apply(clean_supplier_name)
    print("✅ Supplier names standardized.")

    # 3. CLEAN THE MONEY COLUMN
    def clean_money(value):
        # Handle cases where value might be NaN (missing)
        if pd.isna(value):
            return 0.0
        # Remove '$', ',', and spaces. Some numbers might be in parentheses for negative.
        clean_val = str(value).replace('$', '').replace(',', '').strip().replace('(', '-').replace(')', '')
        try:
            return float(clean_val)
        except ValueError:
            # If conversion fails, return 0.0 or log an error
            return 0.0

    df['Clean_Value'] = df['Value (AUD)'].apply(clean_money)
    print("✅ Money values standardized.")

    # --- PACE MODULE 2: BASIC ANALYSIS ---

    # Analysis A: Total Spend by the Consolidated PwC Entity
    pwc_data = df[df['Clean_Supplier'] == 'PwC (Consolidated)']
    total_pwc = pwc_data['Clean_Value'].sum()

    # Analysis B: The "Vague Description" Hunter
    # Looking for titles exactly matching "Management advisory services"
    # Added .fillna('') to handle potential NaN values in 'Title'
    vague_data = df[df['Title'].fillna('').str.lower() == 'management advisory services']
    total_vague = vague_data['Clean_Value'].sum()
    vague_count = len(vague_data)

    # --- REPORTING ---
    print("\n" + "="*40)
    print("      PACE ENGINE - INITIAL REPORT      ")
    print("="*40)
    print(f"💰 Total Spend with PwC (Consolidated): ${total_pwc:,.2f}")
    print(f"🌫️  Contracts with Vague Descriptions:   {vague_count}")
    print(f"💸 Total Value of Vague Contracts:      ${total_vague:,.2f}")
    print("="*40)

    # Optional: Show the first few vague rows to prove it works
    print("\nSample of Vague Contracts Found:")
    # Only show if there are actual vague contracts
    if not vague_data.empty:
        print(vague_data[['Agency', 'Title', 'Value (AUD)']].head())
    else:
        print("No vague contracts found in this sample.")

else:
    print("\n⚠️ Still didn't find the expected column names. Adjust 'skiprows' or check your CSV file.")


✅ Success! Data loaded correctly.
Total rows found: 49
Columns found: ['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12']

First 5 rows of actual data (including headers):
     Unnamed: 0                                         Unnamed: 1  \
0         CN ID                                              Title   
1  CN3820600-A2  Project Assurance Services for the Schools Uni...   
2     CN3922991                       Management advisory services   
3     CN3903240  Payroll voluntary redundancy advisory services...   
4     CN3900356              Supplier Accreditation System Support   

                                          Unnamed: 2    Unnamed: 3  \
0                                             Agency  Publish Date   
1                            Department of Education     15-Oct-21   
2  Office of the Official Secretary to the Govern...      8-Nov

In [ ]:
# --- PACE MODULE 1 (V2): DATA INGESTION & CLEANING ---

import pandas as pd

# 1. LOAD THE DATA with SKIPPING
# We use 'skiprows' to jump over the "Criteria Summary" section at the top.
# If this doesn't work, we might need to change 14 to 15 or 16.
try:
    # Try skipping 14 rows first (common for AusTender)
    df = pd.read_csv('pace_data.csv', encoding='ISO-8859-1', skiprows=14)

    # Double check if we hit the right header row by looking for "Supplier Name"
    if 'Supplier Name' not in df.columns:
        # If not, try reloading with 15 rows skipped (sometimes it varies by 1 line)
        df = pd.read_csv('pace_data.csv', encoding='ISO-8859-1', skiprows=15)

    print("✅ Success! Data loaded correctly.")
    print(f"Total rows found: {len(df)}")
    print("Columns found:", list(df.columns)) # Printing this to be sure

except Exception as e:
    print(f"❌ Error loading data: {e}")

# Only proceed if we actually have the data loaded with correct headers
if 'Supplier Name' in df.columns:

    # 2. CLEAN THE SUPPLIER NAMES (The "PwC" Fix)
    def clean_supplier_name(name):
        name_str = str(name).upper()
        if 'PWC' in name_str or 'PRICEWATERHOUSE' in name_str:
            return 'PwC (Consolidated)'
        else:
            return name

    df['Clean_Supplier'] = df['Supplier Name'].apply(clean_supplier_name)
    print("✅ Supplier names standardized.")

    # 3. CLEAN THE MONEY COLUMN
    def clean_money(value):
        # Remove '$' and ',' and spaces
        clean_val = str(value).replace('$', '').replace(',', '').strip()
        try:
            return float(clean_val)
        except:
            return 0.0

    df['Clean_Value'] = df['Value (AUD)'].apply(clean_money)
    print("✅ Money values standardized.")

    # --- PACE MODULE 2: BASIC ANALYSIS ---

    # Analysis A: Total Spend by the Consolidated PwC Entity
    pwc_data = df[df['Clean_Supplier'] == 'PwC (Consolidated)']
    total_pwc = pwc_data['Clean_Value'].sum()

    # Analysis B: The "Vague Description" Hunter
    # Looking for titles exactly matching "Management advisory services"
    vague_data = df[df['Title'].str.lower() == 'management advisory services']
    total_vague = vague_data['Clean_Value'].sum()
    vague_count = len(vague_data)

    # --- REPORTING ---
    print("\n" + "="*40)
    print("      PACE ENGINE - INITIAL REPORT      ")
    print("="*40)
    print(f"💰 Total Spend with PwC (Consolidated): ${total_pwc:,.2f}")
    print(f"🌫️  Contracts with Vague Descriptions:   {vague_count}")
    print(f"💸 Total Value of Vague Contracts:      ${total_vague:,.2f}")
    print("="*40)

else:
    print("\n⚠️ Still didn't find 'Supplier Name'. Check the skiprows number.")


✅ Success! Data loaded correctly.
Total rows found: 50
Columns found: ['Consultancy?', 'All', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12']

⚠️ Still didn't find 'Supplier Name'. Check the skiprows number.


In [ ]:
import pandas as pd

# Load the data again
df = pd.read_csv('pace_data.csv', encoding='ISO-8859-1')

# Print the list of column names
print("Here are the EXACT column names in your file:")
print(list(df.columns))


Here are the EXACT column names in your file:
['Contract Notice List', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12']


In [ ]:
# --- PACE MODULE 1: DATA INGESTION & CLEANING ---

import pandas as pd
import io

# 1. LOAD THE DATA
# We are telling Python to read the CSV file you uploaded.
# Note: 'encoding' handles special characters often found in gov data.
try:
    df = pd.read_csv('pace_data.csv', encoding='ISO-8859-1')
    print("✅ Success! Data loaded.")
    print(f"Total rows found: {len(df)}")
except FileNotFoundError:
    print("❌ Error: Could not find 'pace_data.csv'. Did you upload it to the folder on the left?")

# 2. CLEAN THE SUPPLIER NAMES (The "PwC" Fix)
def clean_supplier_name(name):
    # Make sure the name is text (string) and make it uppercase
    name_str = str(name).upper()

    # The Logic: If it sounds like PwC, make it "PwC (Consolidated)"
    if 'PWC' in name_str or 'PRICEWATERHOUSE' in name_str:
        return 'PwC (Consolidated)'
    # We can add Deloitte/KPMG later
    else:
        return name

# Apply this logic to a new column
if 'df' in locals(): # Only run if data loaded
    df['Clean_Supplier'] = df['Supplier Name'].apply(clean_supplier_name)
    print("✅ Supplier names standardized.")

    # 3. CLEAN THE MONEY COLUMN
    # Excel money often looks like "$2,500.00". Computers hate the '$' and ','.
    # We need to turn it into a pure number.
    def clean_money(value):
        clean_val = str(value).replace('$', '').replace(',', '')
        try:
            return float(clean_val)
        except:
            return 0.0

    df['Clean_Value'] = df['Value (AUD)'].apply(clean_money)
    print("✅ Money values standardized.")

    # --- PACE MODULE 2: BASIC ANALYSIS ---

    # Analysis A: Total Spend by the Consolidated PwC Entity
    pwc_data = df[df['Clean_Supplier'] == 'PwC (Consolidated)']
    total_pwc = pwc_data['Clean_Value'].sum()

    # Analysis B: The "Vague Description" Hunter
    # Looking for titles exactly matching "Management advisory services"
    vague_data = df[df['Title'].str.lower() == 'management advisory services']
    total_vague = vague_data['Clean_Value'].sum()
    vague_count = len(vague_data)

    # --- REPORTING ---
    print("\n" + "="*40)
    print("      PACE ENGINE - INITIAL REPORT      ")
    print("="*40)
    print(f"💰 Total Spend with PwC (Consolidated): ${total_pwc:,.2f}")
    print(f"🌫️  Contracts with Vague Descriptions:   {vague_count}")
    print(f"💸 Total Value of Vague Contracts:      ${total_vague:,.2f}")
    print("="*40)

    # Optional: Show the first few vague rows to prove it works
    print("\nSample of Vague Contracts Found:")
    print(vague_data[['Agency', 'Title', 'Value (AUD)']].head())


✅ Success! Data loaded.
Total rows found: 65


KeyError: 'Supplier Name'